In [1]:
import os

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [2]:
assert os.environ["OPENAI_API_KEY"]

In [3]:
chunks_with_metadata = []
for text in [
    {"category": "fix", "source": "lammps_fix_nvt.txt"},
    {"category": "pair_style", "source": "lammps_pair_style.txt"},
]:
    loader = TextLoader(text["source"])
    documents = loader.load()
    text_splitter = CharacterTextSplitter(
        chunk_size=1000, chunk_overlap=100, separator="\n"
    )
    chunks = text_splitter.split_documents(documents)
    chunks_with_metadata += chunks

In [4]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(
    documents=chunks_with_metadata, embedding=embeddings, persist_directory="./data"
)

/tmp/ipykernel_25829/2979001685.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [7]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0, model_name="gpt-4o"),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

In [8]:
query = "LAMMPSで圧浴としてMTKを用いたNPTアンサンブルでシミュレーションを行うコードを書いてください。"
result = qa_chain({"query": query})

/tmp/ipykernel_25829/3641214514.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


In [9]:
result

{'query': 'LAMMPSで圧浴としてMTKを用いたNPTアンサンブルでシミュレーションを行うコードを書いてください。',
 'result': '以下は、LAMMPSでMTK（Martyna-Tobias-Klein）を用いたNPTアンサンブルでシミュレーションを行うための基本的なスクリプトの例です。このスクリプトは、シミュレーションボックスの圧力と温度を制御するためにNose-Hooverスタイルのサーモスタットとバロスタットを使用します。\n\n```lammps\n# LAMMPS input script for NPT ensemble using MTK barostat\n\nunits           metal\ndimension       3\nboundary        p p p\natom_style      atomic\n\n# Read in initial configuration\nread_data       data.lammps\n\n# Define interatomic potential\npair_style      eam\npair_coeff      * * potential_file.eam\n\n# Define simulation settings\nneighbor        2.0 bin\nneigh_modify    delay 5\n\n# Define the temperature and pressure\nvariable        T equal 300.0\nvariable        P equal 1.0\n\n# Define the timestep\ntimestep        0.001\n\n# Fix for NPT ensemble\nfix             1 all npt temp ${T} ${T} 0.1 iso ${P} ${P} 1.0\n\n# Output settings\nthermo          100\nthermo_style    custom step temp press vol\n\n# Run the simulation\nrun             1